In [ ]:
import pandas as pd 
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler,  OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numerical_features = [col for col in X.columns if col not in ["Sex"]]
categorical_features = ["Sex"]

# Create a ColumnTransformer to apply different preprocessing strategies
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), categorical_features),
    ],
).fit(X_train)

poly = PolynomialFeatures(3)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

poly.fit(X_train)

X_train = poly.transform(X_train)
X_test = poly.transform(X_test)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Define a neural network model
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
input_dim = X_train_tensor.shape[1]
model = NeuralNet(input_dim)

# Set loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train model
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

# Evaluate model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

y_pred = y_pred_tensor.numpy()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

In [ ]:
import optuna
from sklearn.metrics import mean_squared_log_error

# Define an objective function for Optuna
def objective(trial):
    # Suggest values for hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    hidden1 = trial.suggest_int("hidden1", 32, 128)
    hidden2 = trial.suggest_int("hidden2", 16, 64)
    
    # Define model with trial-selected parameters
    class NeuralNet(nn.Module):
        def __init__(self, input_dim, hidden1, hidden2):
            super(NeuralNet, self).__init__()
            self.fc1 = nn.Linear(input_dim, hidden1)
            self.fc2 = nn.Linear(hidden1, hidden2)
            self.fc3 = nn.Linear(hidden2, 1)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.fc3(x)
            return x
    
    model = NeuralNet(X_train_tensor.shape[1], hidden1, hidden2)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    # Train model
    epochs = 500
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        predictions = model(X_train_tensor)
        loss = criterion(predictions, y_train_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
    
    # Evaluate performance
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
    
    mse = mean_squared_error(y_test_tensor.numpy(), y_pred_tensor.numpy())
    return mse

# Create study and optimize hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Best parameters found
print(f"Best hyperparameters: {study.best_params}")


In [ ]:
best_params = study.best_params

# Define final model using best parameters
class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model with best parameters
input_dim = X_train_tensor.shape[1]
model = NeuralNet(input_dim, best_params["hidden1"], best_params["hidden2"])
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()

# Train final model
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

# Generate predictions using trained model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

y_pred = y_pred_tensor.numpy()

# Calculate R² score
r2 = r2_score(y_test, y_pred)
print(f"Final R² Score: {r2:.4f}")


In [ ]:
df = pd.read_csv("data/train.csv")
X=df.drop(columns=["id","Calories"])
y=df['Calories']
X_submission = pd.read_csv("data/test.csv")
out=X_submission[["id"]]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(), categorical_features),
    ],
).fit(X)

poly = PolynomialFeatures(3)

X = preprocessor.transform(X)
X_submission = preprocessor.transform(X_submission)

poly.fit(X)

X = poly.transform(X)
X_submission = poly.transform(X_submission)


X_train_tensor = torch.tensor(X, dtype=torch.float32)
X_submission = torch.tensor(X_submission, dtype=torch.float32)
y_train_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)


best_params = study.best_params

# Define final model using best parameters
class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model with best parameters
input_dim = X_train_tensor.shape[1]
model = NeuralNet(input_dim, best_params["hidden1"], best_params["hidden2"])
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()

# Train final model
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f'Epoch {epoch}: Loss = {loss.item():.4f}')
# Generate predictions using trained model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_submission)

y_out = y_pred_tensor.numpy()
out["Calories"] = y_out
out.to_csv("data/submissionNN.csv", index=False)
